In [ ]:
from google.colab import drive


drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/ieAI

/content/gdrive/MyDrive/ieAI


In [ ]:
!git clone https://github.com/satchitchatterji/DistillNAM.git

Cloning into 'DistillNAM'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (111/111), done.
remote: Total 150 (delta 50), reused 126 (delta 26), pack-reused 0
Receiving objects: 100% (150/150), 17.63 MiB | 7.27 MiB/s, done.
Resolving deltas: 100% (50/50), done.


In [ ]:
%cd DistillNAM

/content/gdrive/MyDrive/ieAI/DistillNAM


In [ ]:
!pip install -r requirements.txt

/content/gdrive/MyDrive/ieAI/DistillNAM
  Cloning https://github.com/PytorchLightning/lightning-bolts.git (to revision master) to /tmp/pip-install-rsphtz0c/lightning-bolts_f30cc53466824e7e919b200b8aec6316
  Running command git clone --filter=blob:none --quiet https://github.com/PytorchLightning/lightning-bolts.git /tmp/pip-install-rsphtz0c/lightning-bolts_f30cc53466824e7e919b200b8aec6316
  Resolved https://github.com/PytorchLightning/lightning-bolts.git to commit b94ffb0836d8bcc74ccdf0d84b3d42873989b516
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import os
from pprint import pprint
import sys

import pytorch_lightning as L

In [ ]:
# reproducibility
SEED = 42
L.seed_everything(SEED)

INFO:lightning_fabric.utilities.seed:Global seed set to 42


42

In [ ]:
nam_lib_path = os.path.join(os.getcwd(), "nam")
print(f"Insert {nam_lib_path}, sys.path is")
pprint(sys.path)

Insert /content/gdrive/MyDrive/ieAI/DistillNAM/nam, sys.path is
['/content/gdrive/MyDrive/ieAI/DistillNAM/nam',
 '/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/tmp/tmpknjk6xky']


In [ ]:
sys.path.insert(0, nam_lib_path)
pprint(sys.path)

['/content/gdrive/MyDrive/ieAI/DistillNAM/nam',
 '/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/tmp/tmpknjk6xky']


In [ ]:
import utils

mnist_dm = utils.get_MNIST(download_dir="./")
mnist_dm

Datasets root: ./ batch size: 256 n_workers: 1


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<pl_bolts.datamodules.mnist_datamodule.MNISTDataModule>

In [ ]:
# load the CNN teacher
import torch

CNN_ckpt_dict = torch.load("/content/gdrive/MyDrive/ieAI/DistillNAM/MNIST_CNN/lightning_logs/version_2/checkpoints/epoch=9-step=7500.ckpt")
print(CNN_ckpt_dict.keys())
CNN_ckpt_dict["state_dict"].keys()

dict_keys(['epoch', 'global_step', 'pytorch-lightning_version', 'state_dict', 'loops', 'callbacks', 'optimizer_states', 'lr_schedulers'])


odict_keys(['model.conv1.0.weight', 'model.conv1.0.bias', 'model.conv2.0.weight', 'model.conv2.0.bias', 'model.classifier.weight', 'model.classifier.bias'])

In [ ]:
import models

teacher = models.LitMNIST()
teacher.load_state_dict(CNN_ckpt_dict['state_dict'])
teacher

LitMNIST(
  (model): MNISTCnn(
    (conv1): Sequential(
      (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (conv2): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (dropout): Dropout(p=0.5, inplace=False)
    (classifier): Linear(in_features=1600, out_features=10, bias=True)
  )
)

In [ ]:
from torch.nn import functional as F

from nam.wrapper.wrapper import MultiTaskNAMClassifier

student = MultiTaskNAMClassifier(
    num_learners=1,
    patience=60,
    num_epochs=100,
    num_subnets=1,
    metric="accuracy",
    monitor_loss=True,
    early_stop_mode="max",
    n_jobs=1,
    random_state=SEED,
    loss_func=F.cross_entropy,
)
student

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<nam.wrapper.wrapper.MultiTaskNAMClassifier>

In [ ]:
# NOTE these 2 calls are not needed by a L.Trainer, they are here to access the
# dataset manually
mnist_dm.prepare_data()
mnist_dm.setup()
mnist_dm.dataset_train.dataset

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Dataset MNIST
    Number of datapoints: 60000
    Root location: ./
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

In [ ]:
example_pred = student.predict(mnist_dm.dataset_train.dataset.data[0])
example_pred

/content/gdrive/MyDrive/ieAI/DistillNAM/nam/nam/wrapper/wrapper.py:171: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, requires_grad=False, dtype=torch.float)


AttributeError: ignored